# Data Cleaning & Preparation

## Lyme Disease by State data

Since this data only came with state specification, but the climatic variables were only available by Region, I had to manually map each state to one of the four U.S. regions (Northeastern/Eastern, Central/Midwest, Southern, and Western) to which it belongs. In the cell below, I create the mapping function and apply it to the dataset, as well as other general cleaning tasks to ensure compatible data with other sets. 

In [3]:
import pandas as pd

def create_region_mapping():
    return {
        # Northeast/Eastern Region
        'Maine': 'eastern',
        'New Hampshire': 'eastern',
        'Vermont': 'eastern',
        'Massachusetts': 'eastern',
        'Rhode Island': 'eastern',
        'Connecticut': 'eastern',
        'New York': 'eastern',
        'Pennsylvania': 'eastern',
        'New Jersey': 'eastern',
        'Delaware': 'eastern',
        'Maryland': 'eastern',
        'District of Columbia': 'eastern',
        
        # Southern Region
        'Virginia': 'southern',
        'West Virginia': 'southern',
        'Kentucky': 'southern',
        'Tennessee': 'southern',
        'North Carolina': 'southern',
        'South Carolina': 'southern',
        'Georgia': 'southern',
        'Florida': 'southern',
        'Alabama': 'southern',
        'Mississippi': 'southern',
        'Louisiana': 'southern',
        'Arkansas': 'southern',
        
        # Central Region
        'Ohio': 'central',
        'Indiana': 'central',
        'Illinois': 'central',
        'Michigan': 'central',
        'Wisconsin': 'central',
        'Minnesota': 'central',
        'Iowa': 'central',
        'Missouri': 'central',
        'North Dakota': 'central',
        'South Dakota': 'central',
        'Nebraska': 'central',
        'Kansas': 'central',
        
        # Western Region
        'Montana': 'western',
        'Idaho': 'western',
        'Wyoming': 'western',
        'Colorado': 'western',
        'New Mexico': 'western',
        'Arizona': 'western',
        'Utah': 'western',
        'Nevada': 'western',
        'California': 'western',
        'Oregon': 'western',
        'Washington': 'western',
        'Alaska': 'western',
        'Hawaii': 'western'
    }

# apply mapping, clean state names, and melt the dataframe 
def clean_lyme_data(df):
    df_clean = df.copy()
    
    # Clean state names --> remove unusual characters (the original dataset randomly 
    # had some weird characters appended to the end of the state name)
    df_clean['State'] = df_clean['State'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
    
    # melt the dataframe --> convert years to rows
    df_melted = pd.melt(
        df_clean,
        id_vars=['State'],
        var_name='Year',
        value_name='Lyme_cases'
    )
    
    # region mapping
    region_mapping = create_region_mapping()
    
    # add region column
    df_melted['region'] = df_melted['State'].map(region_mapping)
    
    # convert Year to integer
    df_melted['Year'] = pd.to_numeric(df_melted['Year'])
    
    # sort by region, state, and year
    df_melted = df_melted.sort_values(['region', 'State', 'Year'])

    df_melted['Lyme_cases'] = df_melted['Lyme_cases'].str.replace(',', '').astype(int)
    
    return df_melted

df = None

# try different encodings until one works (to remove the weird characters)
encodings_to_try = ['latin1', 'cp1252', 'iso-8859-1', 'utf-8']
for encoding in encodings_to_try:
    try:
        df = pd.read_csv('../data/raw_data/lyme_states_2008-2022_WIDE.csv', encoding=encoding)
        print(f"Successfully read file with {encoding} encoding")
        break
    except UnicodeDecodeError:
        print(f"Failed with {encoding} encoding")
        continue
    except Exception as e:
        print(f"Different error with {encoding} encoding: {str(e)}")
        continue

# apply cleaning function to the data
state_lyme = clean_lyme_data(df)

print("\nFirst few rows of the cleaned dataset:")
print(state_lyme.head())

print("\nUnique state names after cleaning:")
print(sorted(state_lyme['State'].unique()))

print("\nSummary of data by region:")
print(state_lyme.groupby('region')['State'].nunique().to_frame('Number of States'))

Successfully read file with latin1 encoding

First few rows of the cleaned dataset:
        State  Year  Lyme_cases   region
13   Illinois  2008         108  central
65   Illinois  2009         136  central
117  Illinois  2010         135  central
169  Illinois  2011         194  central
221  Illinois  2012         204  central

Unique state names after cleaning:
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'US Total', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',

## Climate data

For this data, I had to download 12 CSV files -- one for each of the three climatic variables of interest (average temperature, minimum temperature, and average precipitation) for each of the four regions. To clean this data, I wrote function for each climatic variable that could loop over each of the regional datasets for that variable. I had to make sure when I was downloading the datasets to name the files consistently. All files began with the region name in lowercase (southern, eastern, central, western) followed by an underscore and then the climatic variable, denoted as either _avgtemp.csv, _mintemp.csv, or _precipitation.csv. This naming consistency allowed me to loop over all files with similar endings, and also extract the region name before the underscore and add it as a column in order to identify the data's region when merged with the other data. 

In [7]:
import pandas as pd
import glob
import os

# function to clean the Average Temperature data 
def clean_avgtemp_data():
    
    avg_dfs = []
    
    # find all avgtemp data for each region
    avgtemp_files = glob.glob("../data/raw_data/*avgtemp.csv")  
    for file in avgtemp_files:
        print(file)
    
    # Process each avgtemp file for the regions
    for file_path in avgtemp_files:
        print(f"\nProcessing {file_path}")
        # Extract region from filename
        region = os.path.basename(file_path).split('_')[0]
        print(f"Extracted region: {region}")
        
        try:
            # Read CSV file (skip first 4 rows, which contain descriptive information)
            df = pd.read_csv(file_path, skiprows=4)
            print(f"Read {len(df)} rows from file")
            
            # Clean and rename necessary columns
            df = df[['Date', 'Value']] 
            df = df.rename(columns={
                'Date': 'Year',
                'Value': 'Avg_temp'
            })
            
            # extract only year from the "date" column 
            df['Year'] = df['Year'].astype(str).str[:4].astype(int)
            
            # add region column
            df['region'] = region
            
            avg_dfs.append(df)

        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    # Combine all regions
    if avg_dfs:
        return pd.concat(avg_dfs, ignore_index=True)
    return None


# Function to clean minimum temperature data 
def clean_mintemp_data():
    min_dfs = []
    
    # find the mintemp files for each region 
    mintemp_files = glob.glob("../data/raw_data/*mintemp.csv") 
    for file in mintemp_files:
        print(file)
    
    # Process each mintemp file for the regions
    for file_path in mintemp_files:
        print(f"\nProcessing {file_path}")
        # Extract region from filename (I had to specify the region name when I downloaded and saved the data)
        region = os.path.basename(file_path).split('_')[0]
        print(f"Extracted region: {region}")
        
        try:
            # Read CSV file (skip the first 4 rows)
            df = pd.read_csv(file_path, skiprows=4)
            print(f"Read {len(df)} rows from file")
            
            # Clean and rename necessary columns
            df = df[['Date', 'Value']] 
            df = df.rename(columns={
                'Date': 'Year',
                'Value': 'Min_temp_avg'
            })
            
            # Extract year from date column
            df['Year'] = df['Year'].astype(str).str[:4].astype(int)
            
            # Add region column
            df['region'] = region
            
            min_dfs.append(df)
        
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    # Combine all regions
    if min_dfs:
        return pd.concat(min_dfs, ignore_index=True)
    return None


# Function to clea precipitation data
def clean_precipitation_data():
    # List to store dataframes
    prec_dfs = []
    
    # Show which files we're finding
    precipitation_files = glob.glob("../data/raw_data/*precipitation.csv")  # Removed underscore from pattern
    for file in precipitation_files:
        print(file)
    
    # Process each mintemp file for the regions
    for file_path in precipitation_files:
        print(f"\nProcessing {file_path}")
        # Extract region from filename
        region = os.path.basename(file_path).split('_')[0]
        print(f"Extracted region: {region}")
        
        try:
            # Read CSV file, skipping the first 4 rows
            df = pd.read_csv(file_path, skiprows=4)
            print(f"Read {len(df)} rows from file")
            
            # Clean and rename columns
            df = df[['Date', 'Value']]  # Keep only needed columns
            df = df.rename(columns={
                'Date': 'Year',
                'Value': 'Precipitation_avg'
            })
            
            # Extract year from the date column
            df['Year'] = df['Year'].astype(str).str[:4].astype(int)
            
            # Add region column
            df['region'] = region
            
            prec_dfs.append(df)
        
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    # Combine all regions
    if prec_dfs:
        return pd.concat(prec_dfs, ignore_index=True)
    return None

# Process both types of data
print("Processing average temperature data...")
avg_temp_data = clean_avgtemp_data()
print("\nProcessing minimum temperature data...")
min_temp_data = clean_mintemp_data()
print("\nProcessing processing data...")
precipitation_data = clean_precipitation_data()

Processing average temperature data...
../data/raw_data/western_avgtemp.csv
../data/raw_data/central_avgtemp.csv
../data/raw_data/eastern_avgtemp.csv
../data/raw_data/southern_avgtemp.csv

Processing ../data/raw_data/western_avgtemp.csv
Extracted region: western
Read 21 rows from file

Processing ../data/raw_data/central_avgtemp.csv
Extracted region: central
Read 21 rows from file

Processing ../data/raw_data/eastern_avgtemp.csv
Extracted region: eastern
Read 21 rows from file

Processing ../data/raw_data/southern_avgtemp.csv
Extracted region: southern
Read 21 rows from file

Processing minimum temperature data...
../data/raw_data/western_mintemp.csv
../data/raw_data/southern_mintemp.csv
../data/raw_data/eastern_mintemp.csv
../data/raw_data/central_mintemp.csv

Processing ../data/raw_data/western_mintemp.csv
Extracted region: western
Read 15 rows from file

Processing ../data/raw_data/southern_mintemp.csv
Extracted region: southern
Read 15 rows from file

Processing ../data/raw_data/ea

## Tree coverage loss data

In [ ]:
tcloss = pd.read_csv('../data/raw_data/treecoverlossdata.csv')

# filter for threshold = 75 and then remove the threshold column
filtered_data = tcloss[tcloss['threshold'] == 75]

# remove other unnecessary columns
columns_to_remove = ['country', 'extent_2000_ha', 'extent_2010_ha', 'gain_2000-2020_ha', 'threshold']
filtered_data = filtered_data.drop(columns=columns_to_remove)

# melt on tc_loss_ha_20** columns
melted_tc = pd.melt(
    filtered_data,
    id_vars=['subnational1', 'area_ha'],
    value_vars=[col for col in filtered_data.columns if col.startswith('tc_loss_ha_')],
    var_name='Year',
    value_name='Tree_Cover_Loss'
)

# clean the 'Year' column to retain only the numeric year
melted_tc['Year'] = melted_tc['Year'].str.extract(r'(\d{4})')
melted_tc['Year'] = pd.to_numeric(melted_tc['Year'])

# rename columns
melted_tc = melted_tc.rename(columns={'subnational1': 'State', 'area_ha': 'Total_Land_Area'})

print(melted_tc.head()) 



        State  Total_Land_Area  Year  Tree_Cover_Loss
0     Alabama         13363464  2001           168587
1      Alaska        150737804  2001            27964
2     Arizona         29535713  2001              653
3    Arkansas         13769059  2001           110114
4  California         40961694  2001            39102


## Species richness data

In [9]:
species = pd.read_csv('../data/raw_data/species_richness_by_state.csv')
species = species.rename(columns={'state': 'State'})

# convert the percentage input to numeric 
species['state_park_land_coverage'] = species['state_park_land_coverage'].str.replace('%', '')
species['state_park_land_coverage'] = pd.to_numeric(species['state_park_land_coverage']) / 100

print(species.head())

        State  species_richness  state_park_land_coverage  state_park_rank
0     Alabama              10.0                    0.0023               46
1      Alaska               NaN                    0.0910                2
2     Arizona               5.0                    0.0260               12
3    Arkansas               8.0                    0.0018               44
4  California              10.0                    0.0749                3


### Merge all datasets

In [10]:
merged = min_temp_data.merge(avg_temp_data, on=['Year', 'region'], how='left')
merged2 = precipitation_data.merge(merged, on=['Year', 'region'], how='outer')
merged3 = merged2.merge(state_lyme, on=['Year', 'region'], how='left')
merged4 = merged3.merge(melted_tc, on=['Year', 'State'], how="left")
merged5 = merged4.merge(species, on='State', how='left')
print(merged5.head())

   Year  Precipitation_avg   region  Min_temp_avg  Avg_temp     State  \
0  2008               4.49  central          53.8      66.1  Illinois   
1  2008               4.49  central          53.8      66.1   Indiana   
2  2008               4.49  central          53.8      66.1      Iowa   
3  2008               4.49  central          53.8      66.1    Kansas   
4  2008               4.49  central          53.8      66.1  Michigan   

   Lyme_cases  Total_Land_Area  Tree_Cover_Loss  species_richness  \
0         108         15008781              615               6.0   
1          42          9436269             1051               8.0   
2         109         14584483              209               5.0   
3          16         21312413              460               1.0   
4          92         25036039            27919               3.0   

   state_park_land_coverage  state_park_rank  
0                    0.0139             19.0  
1                    0.0081             32.0  
2    

In [11]:
merged5.dtypes

Year                          int64
Precipitation_avg           float64
region                       object
Min_temp_avg                float64
Avg_temp                    float64
State                        object
Lyme_cases                    int64
Total_Land_Area               int64
Tree_Cover_Loss               int64
species_richness            float64
state_park_land_coverage    float64
state_park_rank             float64
dtype: object

In [12]:
merged5.to_csv('../data/clean_data/data.csv', index=False)

## Cleaning RCP 8.5 future climate scenarios data

In [14]:
# load the manually compiled dataset
rcp = pd.read_csv('../data/raw_data/RCP8.5_data.csv')

# pivot wider so that each indicator is its own column
rcp = rcp.pivot(index='region', columns='Indicator', values='value').reset_index()

# extract region aggregates from the original dataset
region_aggregates = merged5.groupby('region').agg({
    'Total_Land_Area': 'sum',  # Sum for total land area
    'state_park_land_coverage': 'mean',  # Average for park land coverage
    'state_park_rank': 'mean'  # Average for park rank
}).reset_index()

# merge
future_data = pd.merge(rcp, region_aggregates, on='region')

print(future_data)

future_data.to_csv('../data/clean_data/2050inputs.csv')

     region  Avg_temp  Min_temp  Precipitation_avg  Total_Land_Area  \
0   central     79.33     67.81              12.41       3198039870   
1   eastern     72.39     61.21              12.76        710057100   
2  southern     86.36     75.29              12.58       2105379510   
3   western     74.49     59.00               7.50       6895077150   

   state_park_land_coverage  state_park_rank  
0                  0.008292        31.083333  
1                  0.027827        17.363636  
2                  0.012292        30.916667  
3                  0.032108        19.923077  
